# Training a speech-to-text model using Huggingsound

<a target="_blank" href="https://colab.research.google.com/github/Koffair/colab_pipelines/blob/main/notebooks/03_train_s2t.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**WARNING**
- Open the notebook in Colab by clicking on the "Open in Colab" badge at the top of the notebook
- Save a copy of the notebook to your Google Drive by clicking "File" > "Save a Copy to Drive"
- This notebook assumes that you have downloaded the necessery data and preprocessed it by following the instructions in ```01_train_language_models.ipynb```
